In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

This a quick implementation of a basic deterministic capacity expansion problem using the gurobi bindings for python. Specifically, we solved ![this](https://i.imgur.com/T5BKZ2j.png) with variables given by ![that](https://i.imgur.com/M6Hx2rW.png) this actually just a port of an old homework problem originally done in Julia using the juMP library.

In [2]:
#Test problem data

#Indices
nI = 25
nJ = 25
nT = 10
I = list(range(nI))
J = list(range(nJ))
T = list(range(nT)) 
T_k = [-1]+T #special index for k to prevent index error in constraint
    
def random_array(low,high,size):
    return np.random.uniform(low,high,size)

#Random cost data for testing
C = random_array(low = 20, high = 200, size = (nI,))
H = random_array(low = 1, high = 4, size = (nI,))
M = random_array(low = 10, high = 40,  size = (nI,))
D = random_array(low = 100, high = 500, size = (nJ,nT))
Q = random_array(low = 5000, high = 10000, size = (nJ,))
F = random_array(low = 3, high = 45, size = (nI,nJ,nT))


In [3]:
model = gp.Model('cep')

#Could specify the objective function + variables all in one but I think this hurts readibility. 
x = model.addVars(I,lb = 0.0, name = "x")
p = model.addVars(I,T, lb = 0.0, name = "p")
k = model.addVars(I,T_k, lb = 0.0,  name = "k")
e = model.addVars(I,J,T, lb = 0.0, name = "e")
u = model.addVars(J,T, lb = 0.0, name = "u")


Academic license - for non-commercial use only - expires 2021-05-17
Using license file C:\Users\Tom\gurobi.lic


In [4]:
model.setObjective(sum(C[i]*x[i] for i in I) +
                   sum(H[i]*k[i,t] + M[i]*p[i,t] for i in I for t in T) +
                   sum(F[i,j,t]*e[i,j,t] for i in I for j in J for t in T) + 
                   sum(Q[j]*u[j,t] for j in J for t in T),GRB.MINIMIZE)

In [5]:
model.addConstrs(k[i,-1] == 0 for i in I)
model.addConstrs(k[i,nT-1] >= 0 for i in I)
model.addConstrs(p[i,t] <= x[i] for i in I for t in T)
model.addConstrs(p[i,t] + k[i,t-1] == sum(e[i,j,t] for j in J) + k[i,t] for i in I for t in T)
model.addConstrs(sum(e[i,j,t] for i in I) == D[j,t] - u[j,t] for j in J for t in T)

model.optimize()
x
if model.status == GRB.OPTIMAL:
    sol = model.getAttr('x',x)
    for i in I:
        if sol[i] >0:
            print("Suppplier " + str(i) + ": " + str(sol[i]))

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 800 rows, 7050 columns and 14050 nonzeros
Model fingerprint: 0xff5f7072
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 5e+02]
Presolve removed 75 rows and 435 columns
Presolve time: 0.03s
Presolved: 725 rows, 6615 columns, 13405 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.8013219e+05   9.475514e+03   0.000000e+00      0s
    1234    2.4289632e+06   0.000000e+00   0.000000e+00      0s

Solved in 1234 iterations and 0.12 seconds
Optimal objective  2.428963174e+06
Suppplier 0: 70.87607099591729
Suppplier 1: 2373.6289366481187
Suppplier 2: 814.9471333898811
Suppplier 5: 458.88106990810496
Suppplier 7: 93.46353590146973
Suppplier 9: 85.61223226595875
Suppplier 11: 1173.3546636868796
Suppplier 